# Variant annotation tables & direction of effect investigation

In [96]:
import os
import csv
import re
from collections import Counter

import pandas as pd

from opentargets_pharmgkb.evidence_generation import ID_COL_NAME
from opentargets_pharmgkb.pandas_utils import read_tsv_to_df

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

## 1. Initial data exploration

In [4]:
data_dir = '/home/april/projects/opentargets/pharmgkb/doe'

In [5]:
# Download new data (2024-05-05)
# !cd {data_dir}

# !wget -q https://api.pharmgkb.org/v1/download/file/data/clinicalAnnotations.zip
# !wget -q https://api.pharmgkb.org/v1/download/file/data/variantAnnotations.zip

# !unzip -jq clinicalAnnotations.zip "*.tsv" -d {data_dir}
# !unzip -jq variantAnnotations.zip "*.tsv" -d {data_dir}

# !rm clinicalAnnotations.zip variantAnnotations.zip

The variant annotations zip file contains 4 new tables, described in the readme as follows:
>* **var_pheno_ann.tsv**: Contains associations in which the variant affects a phenotype, with or without drug
information.
>* **var_drug_ann.tsv**: Contains associations in which the variant affects a drug dose, response, metabolism, etc.
>* **var_fa_ann.tsv**: Contains in vitro and functional analysis-type associations.
>* **study_parameters.tsv**: Contains information about the study population size, biogeographical group and statistics
for the variant annotations; this file is cross-referenced against the 3 variant annotation files.

Study parameters table is interesting but feels out of scope for now at least, will ignore for the rest of the notebook.

In [11]:
var_drug_ann = read_tsv_to_df(os.path.join(data_dir, 'var_drug_ann.tsv'))
var_fa_ann = read_tsv_to_df(os.path.join(data_dir, 'var_fa_ann.tsv'))
var_pheno_ann = read_tsv_to_df(os.path.join(data_dir, 'var_pheno_ann.tsv'))

Questions to consider:
* How many annotations?
* What's the coverage of variant/haplotypes relative to clinical annotations?
* What are the relevant fields?
* What's the relationship between these annotations and clinical annotations?
* Which of these columns has a controlled vocab vs. free text?
* How do the different variant-level annotation sentences contribute to the overall clincial annotation sentences?

In [30]:
len(var_drug_ann)

11901

In [31]:
len(var_fa_ann)

2009

In [32]:
len(var_pheno_ann)

13517

In [26]:
var_drug_ann.head()

,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population,Metabolizer types,isPlural,Is/Is Not associated,Direction of effect,PD/PK terms,Multiple drugs And/or,Population types,Population Phenotypes or diseases,Multiple phenotypes or diseases And/or,Comparison Allele(s) or Genotype(s),Comparison Metabolizer types
0,1451834452,"CYP3A4*1, CYP3A4*17",CYP3A4,nifedipine,15634941,"Other, Metabolism/PK",not stated,in vitro expression of the recombinant CYP3A4*17 allelic protein and the wild-type protein,CYP3A4 *17 is associated with decreased metabolism of nifedipine as compared to CYP3A4 *1.,*17,NaN,NaN,Is,Associated with,decreased,metabolism of,NaN,NaN,NaN,NaN,*1,NaN
1,1451159680,rs5031016,CYP2A6,warfarin,22248286,Dosage,no,No association was found between this variant and warfarin-maintenance dose. Described as CYP2A6*7 in this study.,Allele G is not associated with increased dose of warfarin in people with an international normalized ratio (INR) of 2.0-3.0 as compared to allele A.,G,NaN,NaN,Is,Not associated with,increased,dose of,NaN,in people with,Other:an international normalized ratio (INR) of 2.0-3.0,NaN,A,NaN
2,1451306860,CYP2C9*11,CYP2C9,warfarin,33350885,Dosage,not stated,"""This case suggests that CYP2C9 *11/*11 carriers require approximately two thirds less warfarin than CYP2C9"" normal function homozygotes.",CYP2C9 *11/*11 is associated with decreased dose of warfarin.,*11/*11,NaN,NaN,Is,Associated with,decreased,dose of,NaN,NaN,NaN,NaN,NaN,NaN
3,1448997750,"CYP2B6*1, CYP2B6*18",CYP2B6,efavirenz,16495778,Metabolism/PK,yes,"Please note that in the paper the allele was referred to as CYP2B6*16. CYP2B6*16 and *18 alleles have been consolidated by PharmVar in Jan 2020, with *16 now listed as a suballele of *18 (CYP2B6*18.002). This annotation is updated to be on CYP2B6*18, instead of CYP2B6*16.",CYP2B6 *1/*18 is associated with increased concentrations of efavirenz in people with HIV Infections as compared to CYP2B6 *1/*1.,*1/*18,NaN,NaN,Is,Associated with,increased,concentrations of,NaN,in people with,Disease:HIV Infections,NaN,*1/*1,NaN
4,1448631821,"CYP2C19*1, CYP2C19*2",CYP2C19,"clomipramine, desmethyl clomipramine",28470111,Metabolism/PK,no,in a single individual,CYP2C19 *1/*2 is associated with increased trough concentration of clomipramine and desmethyl clomipramine.,*1/*2,NaN,NaN,Is,Associated with,increased,trough concentration of,and,NaN,NaN,NaN,NaN,NaN


In [27]:
var_fa_ann.head()

,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population,Assay type,Metabolizer types,isPlural,Is/Is Not associated,Direction of effect,Functional terms,Gene/gene product,When treated with/exposed to/when assayed with,Multiple drugs And/or,Cell type,Comparison Allele(s) or Genotype(s),Comparison Metabolizer types
0,1451148445,"CYP2C19*1, CYP2C19*17",CYP2C19,normeperidine,30902024,NaN,not stated,"In other in vitro experiments, normeperidine formation was significantly correlated with CYP2C19 activity, as measured by S-mephenytoin 4-hydroxylation.",CYP2C19 *17/*17 is associated with increased formation of normeperidine as compared to CYP2C19 *1/*1 + *1/*17.,*17/*17,NaN,in human liver microsomes,NaN,Is,Associated with,increased,formation of,NaN,NaN,NaN,NaN,*1/*1 + *1/*17,NaN
1,1447814273,rs9923231,VKORC1,NaN,26847243,Other,no,NaN,Allele T is not associated with transcription of VKORC1 in HepG2 cells as compared to allele C.,T,NaN,luciferase assay,NaN,Is,Not associated with,NaN,transcription of,VKORC1,NaN,NaN,in HepG2 cells,C,NaN
2,1447814277,rs56314408,VKORC1,NaN,26847243,Other,yes,"In the European population, this SNPs is in high LD with rs9923231 but not other populations. This SNP disrupts a binding motif for transcription factor TFAP2A/C.",Allele C is associated with increased transcription of VKORC1 in HepG2 cells as compared to allele T.,C,NaN,luciferase assay,NaN,Is,Associated with,increased,transcription of,VKORC1,NaN,NaN,in HepG2 cells,T,NaN
3,1447990384,rs1065852,CYP2D6,bufuralol,2211621,Metabolism/PK,not stated,In vitro experiments showed a significant decrease in CYP2D6 activity for the variant construct expressed in COS-1 cells as compared to wild-type.,Allele A is associated with decreased activity of CYP2D6 when assayed with bufuralol in COS-1 cells as compared to allele G.,A,NaN,NaN,NaN,Is,Associated with,decreased,activity of,CYP2D6,when assayed with,NaN,in COS-1 cells,G,NaN
4,1448281185,"CYP2B6*1, CYP2B6*6",CYP2B6,bupropion,27439448,Efficacy,yes,The ratio of hydroxybupropion versus bupropion (AUC_hyd/ AUC_bup) in terms of area under the time-concentration curve (AUC) was used to assay CYP2B6 activity.,CYP2B6 *1/*1 is associated with increased activity of CYP2B6 when assayed with bupropion as compared to CYP2B6 *1/*6.,*1/*1,NaN,NaN,NaN,Is,Associated with,increased,activity of,CYP2B6,when assayed with,NaN,NaN,*1/*6,NaN


In [28]:
var_pheno_ann.head()

,Variant Annotation ID,Variant/Haplotypes,Gene,Drug(s),PMID,Phenotype Category,Significance,Notes,Sentence,Alleles,Specialty Population,Metabolizer types,isPlural,Is/Is Not associated,Direction of effect,Side effect/efficacy/other,Phenotype,Multiple phenotypes And/or,When treated with/exposed to/when assayed with,Multiple drugs And/or,Population types,Population Phenotypes or diseases,Multiple phenotypes or diseases And/or,Comparison Allele(s) or Genotype(s),Comparison Metabolizer types
0,1449169911,HLA-B*35:08,HLA-B,lamotrigine,29238301,Toxicity,no,"The allele was not significant when comparing allele frequency in cases of severe cutaneous adverse reactions (SCAR), Stevens-Johnson Syndrome (SJS) and Maculopapular Exanthema (MPE) (1/15) and controls (individuals without AEs who took lamotrigine) (0/50). The allele was significant when comparing between cases (1/15) and the general population (1/986).","HLA-B *35:08 is not associated with likelihood of Maculopapular Exanthema, severe cutaneous adverse reactions or Stevens-Johnson Syndrome when treated with lamotrigine in people with Epilepsy.",*35:08,NaN,NaN,Is,Not associated with,NaN,likelihood of,"Side Effect:Maculopapular Exanthema, Side Effect:severe cutaneous adverse reactions, Side Effect:Stevens-Johnson Syndrome",or,when treated with,NaN,in people with,Disease:Epilepsy,NaN,NaN,NaN
1,982022165,rs45607939,NAT2,sulfamethoxazole / trimethoprim,22850190,Toxicity,no,Minor allele frequencies were compared between cases (with drug-induced hypersensitivity) and controls.,Allele T is not associated with increased risk of Hypersensitivity when treated with sulfamethoxazole / trimethoprim in people with Infection.,T,NaN,NaN,Is,Not associated with,increased,risk of,Disease:Hypersensitivity,NaN,when treated with,NaN,in people with,Disease:Infection,NaN,NaN,NaN
2,982022148,rs1799930,NAT2,sulfamethoxazole / trimethoprim,22850190,Toxicity,no,Minor allele frequencies were compared between cases (with drug-induced hypersensitivity) and controls.,Allele A is not associated with increased risk of Hypersensitivity when treated with sulfamethoxazole / trimethoprim in people with Infection.,A,NaN,NaN,Is,Not associated with,increased,risk of,Disease:Hypersensitivity,NaN,when treated with,NaN,in people with,Disease:Infection,NaN,NaN,NaN
3,1451283480,rs16969968,CHRNA5,NaN,22071378,Other,yes,"this was from meta-analysis of 27 studies but the number of total cases and the risk allele not clearly specified. Minor allele frequency was given for A allele. Introduction states that variant is Asp398Asn, where Asn (A allele) has lower nicotine response than Asp (G allele) and may be at greater risk for nicotine addiction.",Allele A is associated with increased severity of Tobacco Use Disorder in people with Tobacco Use Disorder.,A,NaN,NaN,Is,Associated with,increased,severity of,Other:Tobacco Use Disorder,NaN,NaN,NaN,in people with,Other:Tobacco Use Disorder,NaN,NaN,NaN
4,1444696916,rs267606617,MT-RNR1,streptomycin,7689389,Toxicity,not stated,"Pedigree analysis with 3 separate families. Within the maternal lines, 15 individuals had the 1555G variant, took aminoglycoside antibiotics, and developed hearing loss. 100% of individuals with the 1555G variant who took aminoglycosides developed hearing loss. Homoplasmic. Please note that no statistical analyses were done.",Allele G is associated with Ototoxicity when treated with streptomycin as compared to allele A.,G,NaN,NaN,Is,Associated with,NaN,NaN,Side Effect:Ototoxicity,and,when treated with,NaN,NaN,NaN,NaN,A,NaN


The 3 annotations tables provide evidence for the clinical annotations, can be connected by joining with the `clinical_ann_evidence.tsv` file. In general a clinical annotation can have multiple variant annotations as evidence, and a variant annotation can be used as evidence for multiple clinical annotations.

Each of these tables has a "Direction of effect" column, and the type of "effect" is different for each - likelihood of side effects, formation of product, metabolism of drug, etc.

**Question for OT**: when we say "direction of effect", do we mean any of these "effects"? I.e. should we include all three of these tables or focus on one?

In [38]:
clinical_annotations = read_tsv_to_df(os.path.join(data_dir, 'clinical_annotations.tsv'))
clinical_ann_evidence = read_tsv_to_df(os.path.join(data_dir, 'clinical_ann_evidence.tsv'))
clinical_ann_alleles = read_tsv_to_df(os.path.join(data_dir, 'clinical_ann_alleles.tsv'))

In [122]:
main_df = pd.merge(clinical_annotations, clinical_ann_evidence, how='left', on=ID_COL_NAME)
# main_df = pd.merge(main_df, clinical_ann_alleles, on=ID_COL_NAME)
main_df = main_df[[
    # Main table
    'Clinical Annotation ID', 'Variant/Haplotypes', 'Gene', 'Level of Evidence', 'Phenotype Category', 'Drug(s)', 'Phenotype(s)',
    # Evidence table
    'Evidence ID', 'Evidence Type', 'PMID', 'Summary',
]]

In [123]:
# https://www.pharmgkb.org/clinicalAnnotation/981755803 - has all three types of variant annotation evidence, plus label/guideline evidence
df_981755803 = main_df[main_df[ID_COL_NAME] == '981755803']

In [124]:
# Just look at one clinical annotation as an example
df_981755803_drug = pd.merge(df_981755803, var_drug_ann, left_on='Evidence ID', right_on='Variant Annotation ID', how='inner', suffixes=(None, '_var_drug'))
df_981755803_pheno = pd.merge(df_981755803, var_pheno_ann, left_on='Evidence ID', right_on='Variant Annotation ID', how='inner', suffixes=(None, '_var_pheno'))
df_981755803_fa = pd.merge(df_981755803, var_fa_ann, left_on='Evidence ID', right_on='Variant Annotation ID', how='inner', suffixes=(None, '_var_fa'))

In [125]:
len(df_981755803)

30

In [126]:
len(df_981755803_drug)

24

In [127]:
len(df_981755803_fa)

2

In [128]:
len(df_981755803_pheno)

2

In [85]:
df_981755803_drug

,Clinical Annotation ID,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s),Evidence ID,Evidence Type,PMID,Summary,Variant Annotation ID,Variant/Haplotypes_var_drug,Gene_var_drug,Drug(s)_var_drug,PMID_var_drug,Phenotype Category_var_drug,Significance,Notes,Sentence,Alleles,Specialty Population,Metabolizer types,isPlural,Is/Is Not associated,Direction of effect,PD/PK terms,Multiple drugs And/or,Population types,Population Phenotypes or diseases,Multiple phenotypes or diseases And/or,Comparison Allele(s) or Genotype(s),Comparison Metabolizer types
0,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,981755665,Variant Drug Annotation,21083385,Genotypes AA + AG are associated with response to ivacaftor in people with Cystic Fibrosis.,981755665,rs75527207,CFTR,ivacaftor,21083385,Efficacy,not stated,Clinical trials were carried out to test efficacy of ivacaftor selecting only patients with the CFTR G551D mutation on at least one allele (genotype AA or AG).,Genotypes AA + AG are associated with response to ivacaftor in people with Cystic Fibrosis.,AA + AG,NaN,NaN,Are,Associated with,NaN,response to,NaN,in people with,Disease:Cystic Fibrosis,NaN,NaN,NaN
1,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,981755678,Variant Drug Annotation,22047557,Genotypes AA + AG are associated with response to ivacaftor in people with Cystic Fibrosis.,981755678,rs75527207,CFTR,ivacaftor,22047557,Efficacy,not stated,A clinical trial that selected patients with the G551D CFTR mutation (rs75527207 genotype AA or AG). Patients without this mutation were excluded. One patient included in the placebo group was homozygous for F508del (rs113993960 genotype del/del).,Genotypes AA + AG are associated with response to ivacaftor in people with Cystic Fibrosis.,AA + AG,NaN,NaN,Are,Associated with,NaN,response to,NaN,in people with,Disease:Cystic Fibrosis,NaN,NaN,NaN
2,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,982009991,Variant Drug Annotation,23590265,Allele A is associated with response to ivacaftor in children with Cystic Fibrosis.,982009991,rs75527207,CFTR,ivacaftor,23590265,Efficacy,yes,Patients aged 6-11 at time of screening who had at least one allele with the G551D mutation (allele A at position rs75527207) were recruited for this trial. Ivacaftor is only indicated in CF patients with this mutation. Significant improvements in lung function were seen in the ivacaftor treatment group compared to placebo.,Allele A is associated with response to ivacaftor in children with Cystic Fibrosis.,A,Pediatric,NaN,Is,Associated with,NaN,response to,NaN,in children with,Disease:Cystic Fibrosis,NaN,NaN,NaN
3,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,1183629335,Variant Drug Annotation,24066763,Genotype AA is associated with response to ivacaftor in women with Cystic Fibrosis.,1183629335,rs75527207,CFTR,ivacaftor,24066763,Efficacy,not stated,"Case report of a female homozygous for the G551D CFTR mutation (genotype AA) in which ivacaftor was efficacious: increased absolute change in percent of predicted FEV1, increased weight and walk distance and decreased sweat chloride levels over a 12 month course with no sign of plateau to date.",Genotype AA is associated with response to ivacaftor in women with Cystic Fibrosis.,AA,NaN,NaN,Is,Associated with,NaN,response to,NaN,in women with,Disease:Cystic Fibrosis,NaN,NaN,NaN
4,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,1448423752,Variant Drug Annotation,27773592,Genotypes AA + AG is associated with increased response to ivacaftor in people with Cystic Fibrosis as compared to genotype GG.,1448423752,rs75527207,CFTR,ivacaftor,27773592,Efficacy,yes,The outcome of change in sweat chloride was correlated with change in FEV1 in patients with cystic fibrosis and found to have improved results for both.,Genotypes AA + AG is associated with increased response to ivacaftor in people with Cystic Fibrosis as compared to geno

In [138]:
df_981755803_fa

,Clinical Annotation ID,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s),Evidence ID,Evidence Type,PMID,Summary,Variant Annotation ID,Variant/Haplotypes_var_fa,Gene_var_fa,Drug(s)_var_fa,PMID_var_fa,Phenotype Category_var_fa,Significance,Notes,Sentence,Alleles,Specialty Population,Assay type,Metabolizer types,isPlural,Is/Is Not associated,Direction of effect,Functional terms,Gene/gene product,When treated with/exposed to/when assayed with,Multiple drugs And/or,Cell type,Comparison Allele(s) or Genotype(s),Comparison Metabolizer types
0,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,1043737620,Variant Functional Assay Annotation,23757361,Allele A is associated with increased activity of CFTR when treated with ivacaftor in transfected CHO cells.,1043737620,rs75527207,CFTR,ivacaftor,23757361,Efficacy,yes,compared to no treatment. Ivacaftor stimulated CFTR activity in CFTR-G551D expressing CHO cells (as measured by iodine efflux).,Allele A is associated with increased activity of CFTR when treated with ivacaftor in transfected CHO cells.,A,NaN,NaN,NaN,Is,Associated with,increased,activity of,CFTR,when treated with,NaN,in transfected CHO cells,NaN,NaN
1,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,1043737636,Variant Functional Assay Annotation,23891399,Allele A is associated with activity of CFTR when treated with ivacaftor in FRT cell lines.,1043737636,rs75527207,CFTR,ivacaftor,23891399,Efficacy,yes,G551D allele. 55.3 fold increase in chloride transport upon ivacaftor treatment as compared to baseline (no ivacaftor treatment).,Allele A is associated with activity of CFTR when treated with ivacaftor in FRT cell lines.,A,NaN,NaN,NaN,Is,Associated with,NaN,activity of,CFTR,when treated with,NaN,in FRT cell lines,NaN,NaN


In [94]:
df_981755803_pheno

,Clinical Annotation ID,Variant/Haplotypes,Gene,Level of Evidence,Phenotype Category,Drug(s),Phenotype(s),Evidence ID,Evidence Type,PMID,Summary,Variant Annotation ID,Variant/Haplotypes_var_pheno,Gene_var_pheno,Drug(s)_var_pheno,PMID_var_pheno,Phenotype Category_var_pheno,Significance,Notes,Sentence,Alleles,Specialty Population,Metabolizer types,isPlural,Is/Is Not associated,Direction of effect,Side effect/efficacy/other,Phenotype,Multiple phenotypes And/or,When treated with/exposed to/when assayed with,Multiple drugs And/or,Population types,Population Phenotypes or diseases,Multiple phenotypes or diseases And/or,Comparison Allele(s) or Genotype(s),Comparison Metabolizer types
0,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,1448267532,Variant Phenotype Annotation,27745802,Genotypes AA + AG is associated with decreased severity of bone density when treated with ivacaftor in people with Cystic Fibrosis as compared to genotype GG.,1448267532,rs75527207,CFTR,ivacaftor,27745802,Efficacy,yes,Bone mineral density compared before and after 1 year of treatment with ivacaftor using dual energy X-ray absorptiometry at the L2-L4 lumbar spine. All patients were pancreatic insufficient.,Genotypes AA + AG is associated with decreased severity of bone density when treated with ivacaftor in people with Cystic Fibrosis as compared to genotype GG.,AA + AG,NaN,NaN,Is,Associated with,decreased,severity of,Side Effect:bone density,and,when treated with,NaN,in people with,Disease:Cystic Fibrosis,NaN,GG,NaN
1,981755803,rs75527207,CFTR,1A,Efficacy,ivacaftor,Cystic Fibrosis,1449192031,Variant Phenotype Annotation,28651844,Allele A is associated with decreased likelihood of cystic fibrosis pulmonary exacerbation when treated with ivacaftor in people with Cystic Fibrosis.,1449192031,rs75527207,CFTR,ivacaftor,28651844,Efficacy,yes,G551D allele. Patients receiving ivacaftor treatment had a reduced rate of pulmonary exacerbation events compared to patients receiving a placebo.,Allele A is associated with decreased likelihood of cystic fibrosis pulmonary exacerbation when treated with ivacaftor in people with Cystic Fibrosis.,A,Pediatric,NaN,Is,Associated with,decreased,likelihood of,Disease:cystic fibrosis pulmonary exacerbation,and,when treated with,NaN,in people with,Disease:Cystic Fibrosis,NaN,NaN,NaN


In [93]:
# Comparing number of PMIDs vs. number of evidence
len(set(df_981755803_drug['PMID']) | set(df_981755803_pheno['PMID']) | set(df_981755803_fa['PMID']))

28

### Observations so far
Clinical annotation [981755803](https://www.pharmgkb.org/clinicalAnnotation/981755803) has 30 supporting evidence:
* 24 variant/drug annotations
* 2 variant/functional assay annotations
* 2 variant/phenotype annotations
* 2 others (drug labels & guidelines, present in another data download so not included here)

Each variant annotation is associated with a PMID, these are 1:1.
* We should think about whether we want to preserve the PMID & evidence associations.

These annotations seem much more specific than the clinical annotations, e.g.
* they distinguish between "disease" and "side effect" (check how often)
* if there are multiple phenotypes or drugs, they specify whether these should be "and"s or "or"s

These annotations are specific to one or more alleles or genotypes, so we will need to associate them accordingly.
* Annoyingly multiples are represented in yet another way, e.g. `AA + AG` (check how consistent this is)

It would good if we could select the relevant columns from the 3 variant annotation tables and merge them into a unified representation, so we don't have to manage them separately in the pipelines or in the UI


## 2. Coverage and data consistency

In [97]:
len(clinical_annotations)

5111

In [129]:
# Exploded on evidence - average 3 per annotation
len(main_df)

15129

In [130]:
# Add all the var annotation tables - this will be a huge mess
main_with_var = pd.merge(main_df, var_drug_ann, left_on='Evidence ID', right_on='Variant Annotation ID', how='left', suffixes=(None, '_var_drug'))
main_with_var = pd.merge(main_with_var, var_pheno_ann, left_on='Evidence ID', right_on='Variant Annotation ID', how='left', suffixes=(None, '_var_pheno'))
main_with_var = pd.merge(main_with_var, var_fa_ann, left_on='Evidence ID', right_on='Variant Annotation ID', how='left', suffixes=(None, '_var_fa'))

In [131]:
# sanity check
len(main_with_var)

15129

In [144]:
ca_with_var_evidence = set(main_with_var[main_with_var['Sentence'].notna() | main_with_var['Sentence_var_pheno'].notna() | main_with_var['Sentence_var_fa'].notna()][ID_COL_NAME])

In [147]:
# Every clinical annotation has at least one variant annotation as supporting evidence
len(ca_with_var_evidence)

5111

In [148]:
ca_with_doe_evidence = set(main_with_var[main_with_var['Direction of effect'].notna() | main_with_var['Direction of effect_var_pheno'].notna() | main_with_var['Direction of effect_var_fa'].notna()][ID_COL_NAME])

In [149]:
# Most contain some kind of direction of effect info
len(ca_with_doe_evidence)

4917

Observations so far:
* Every clinical annotation has at least one variant annotation as supporting evidence
* Most contain some kind of direction of effect info (i.e. in one of the three tables)
   * => coverage is good, assuming we care about all three types of effects

In [132]:
main_with_var.columns

Index(['Clinical Annotation ID', 'Variant/Haplotypes', 'Gene',
       'Level of Evidence', 'Phenotype Category', 'Drug(s)', 'Phenotype(s)',
       'Evidence ID', 'Evidence Type', 'PMID', 'Summary',
       'Variant Annotation ID', 'Variant/Haplotypes_var_drug', 'Gene_var_drug',
       'Drug(s)_var_drug', 'PMID_var_drug', 'Phenotype Category_var_drug',
       'Significance', 'Notes', 'Sentence', 'Alleles', 'Specialty Population',
       'Metabolizer types', 'isPlural', 'Is/Is Not associated',
       'Direction of effect', 'PD/PK terms', 'Multiple drugs And/or',
       'Population types', 'Population Phenotypes or diseases',
       'Multiple phenotypes or diseases And/or',
       'Comparison Allele(s) or Genotype(s)', 'Comparison Metabolizer types',
       'Variant Annotation ID_var_pheno', 'Variant/Haplotypes_var_pheno',
       'Gene_var_pheno', 'Drug(s)_var_pheno', 'PMID_var_pheno',
       'Phenotype Category_var_pheno', 'Significance_var_pheno',
       'Notes_var_pheno', 'Sentence_var_

In [133]:
all_var_ann_cols = set(var_drug_ann.columns) | set(var_fa_ann.columns) | set(var_pheno_ann.columns)
common_var_ann_cols = set(var_drug_ann.columns) & set(var_fa_ann.columns) & set(var_pheno_ann.columns)

In [134]:
common_var_ann_cols

{'Alleles',
 'Comparison Allele(s) or Genotype(s)',
 'Comparison Metabolizer types',
 'Direction of effect',
 'Drug(s)',
 'Gene',
 'Is/Is Not associated',
 'Metabolizer types',
 'Multiple drugs And/or',
 'Notes',
 'PMID',
 'Phenotype Category',
 'Sentence',
 'Significance',
 'Specialty Population',
 'Variant Annotation ID',
 'Variant/Haplotypes',
 'isPlural'}

In [135]:
unique_var_ann_cols = all_var_ann_cols - common_var_ann_cols

In [136]:
# annotate with origin table
annotated_unique_var_ann_cols = {'drug':[], 'fa':[], 'pheno':[]}
for c in unique_var_ann_cols:
    if c in var_drug_ann.columns:
        annotated_unique_var_ann_cols['drug'].append(c)
    if c in var_fa_ann.columns:
        annotated_unique_var_ann_cols['fa'].append(c)
    if c in var_pheno_ann.columns:
        annotated_unique_var_ann_cols['pheno'].append(c)

In [137]:
annotated_unique_var_ann_cols

{'drug': ['Multiple phenotypes or diseases And/or',
  'Population types',
  'Population Phenotypes or diseases',
  'PD/PK terms'],
 'fa': ['Cell type',
  'Functional terms',
  'When treated with/exposed to/when assayed with',
  'Gene/gene product',
  'Assay type'],
 'pheno': ['Multiple phenotypes or diseases And/or',
  'Side effect/efficacy/other',
  'Multiple phenotypes And/or',
  'When treated with/exposed to/when assayed with',
  'Population types',
  'Population Phenotypes or diseases',
  'Phenotype']}

* Population phenotype always goes with "multiple phenotypes and/or" (note functional assay doesn't mention phenotype - I guess there's no multiples in the gene product?)
    * where does the main table phenotype come from?? OT is overwriting this now but I'm still confused
* Drug (main table) always goes with "multiple drugs and/or"
* Comparison alleles in theory I guess tells us something about the reference or baseline, doesn't seem frequently filled in though

Sentence examples:
* **drug**: "Genotypes AA + AG are associated with response to ivacaftor in people with Cystic Fibrosis."
    * alleles = "AA + AG"
    * direction of effect = [none]
    * pd/pk term = "response to"
    * drug = "ivacaftor"
    * population types = "people"
    * population phenotype = "cystic fibrosis"
* **fa**: "Allele A is associated with increased activity of CFTR when treated with ivacaftor in transfected CHO cells."
    * alleles = "A"
    * direction of effect = "increased"
    * functional term = "activity of"
    * gene/gene product = "CFTR"
    * when treated with/exposed to/assayed with = "when treated with"
    * drug = "ivacaftor"
    * cell type = "transfected CHO cells"
* **pheno**: "Genotypes AA + AG is associated with decreased severity of bone density when treated with ivacaftor in people with Cystic Fibrosis as compared to genotype GG."
    * alleles = "AA + AG"
    * direction of effect = "decreased"
    * side effect/efficacy/other = "severity of"
    * phenotype = "bone density" **< note distinction between this and population phenotype**
    * when treated with/exposed to/assayed with = "when treated with"
    * drug = "ivacaftor"
    * population types = "people"
    * population phenotype = "cystic fibrosis"

For the simplest direction of effect annotation (i.e. not including the population, cell type, etc.), I think we only strictly need the following:
1. direction of effect
2. pd/pk term | functional term | side effect/efficacy/other
3. drug | gene/gene product | phenotype

This tells us the direction (1) and what the effect is (2&3).
Of course we also need the alleles to associate with the appropriate evidence string, and the "is/is not associated" column so we don't report negative results (unless we want to).

In [161]:
# Check some vocab
set(main_with_var['Direction of effect']) | set(main_with_var['Direction of effect_var_pheno']) | set(main_with_var['Direction of effect_var_fa'])

{'decreased', 'increased', nan}

In [158]:
set(main_with_var['PD/PK terms'])

{'clearance of',
 'clinical benefit to',
 'concentrations of',
 'discontinuation of',
 'dose of',
 'dose-adjusted trough concentrations of',
 'exposure to',
 'half-life time of',
 'metabolism of',
 nan,
 'resistance to',
 'response to',
 'steady-state concentration of',
 'time to response to',
 'trough concentration of'}

In [159]:
set(main_with_var['Functional terms'])

{'activity of',
 'affinity to',
 'catalytic activity of',
 'clearance of',
 'concentrations of',
 'enzyme activity of',
 'expression of',
 'formation of',
 'glucuronidation of',
 'half-life of',
 'inhibition of',
 'metabolism of',
 nan,
 'protein stability of',
 'sensitivity to',
 'steady-state level of',
 'sulfation of',
 'transcription of',
 'transport of',
 'uptake of'}

In [160]:
set(main_with_var['Side effect/efficacy/other'])

{'age at onset of', 'likelihood of', nan, 'risk of', 'severity of'}

Assume the last term will vary but hopefully we can also map it to the relevant domain (CHEMBL, EFO, Ensembl).

This is a pretty small set of terms for ca. 15000 rows, though I don't think we can assume the vocab won't grow (except the actual direction word, we should be good there).

## 3. Direction of effect prototyping

### To-do's

Consistency:
* check consistency of allele/genotype representations - SNPs and star alleles
    * ensure we can connect these to the alleles table
* check consistency of disease/side effect/other? representation for phenotypes
    * ensure we can parse and map these if needed

Prototyping:
* sample schema additions for unified representation
* sample extraction

Specific questions:
* Do we care about all of the tables? (for v1)
* Do we care about the other columns? (for v1)